# Covid Dataset

In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
%matplotlib inline
pd.set_option('display.max_rows', 500)

## lecture des données 

In [84]:
baseCovid = pd.read_csv('./data/baseCovid.csv')

## Suppression des donnees redondantes  

In [85]:
baseCovid.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
      dtype='object')

In [86]:
baseCovid.drop( columns=['dateRep' , 'geoId' ,'countryterritoryCode','continentExp'], axis=1, inplace = True) 

## Traitement des données (vide 'NaN', données negatives)

In [87]:
baseCovid.columns

Index(['day', 'month', 'year', 'cases', 'deaths', 'countriesAndTerritories',
       'popData2019',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
      dtype='object')

In [88]:
print(baseCovid['day'].isna().sum())
print(baseCovid['month'].isna().sum())
print(baseCovid['year'].isna().sum())
print(baseCovid['cases'].isna().sum())
print(baseCovid['deaths'].isna().sum())
print(baseCovid['countriesAndTerritories'].isna().sum())
print(baseCovid['popData2019'].isna().sum())
print(baseCovid['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'].isna().sum())

0
0
0
0
0
0
380
3123


In [89]:
# par raport au nombre totales des lignes c'est negligable, on supprime les lignes contenant 
baseCovid['popData2019'].dropna( )
baseCovid['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'].dropna()

0        9.013779
1        7.052776
2        6.868768
3        7.134266
4        6.968658
           ...   
61882    0.047796
61883    0.047796
61884    0.047796
61885    0.054624
61886    0.054624
Name: Cumulative_number_for_14_days_of_COVID-19_cases_per_100000, Length: 58777, dtype: float64

In [92]:
baseCovid[DataCovid['popData2019'] <0]

,day,month,year,cases,deaths,countriesAndTerritories,popData2019,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000


In [94]:
print(len(baseCovid[baseCovid['cases'] <0]))
print(len(baseCovid[baseCovid['deaths'] <0]))
print(len(baseCovid[baseCovid['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'] <0]))

18
8
40


In [100]:
baseCovid = baseCovid[baseCovid['cases'] >=0]
baseCovid = baseCovid[baseCovid['deaths'] >=0]
baseCovid = baseCovid[baseCovid['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'] >=0]

In [101]:
baseCovid

,day,month,year,cases,deaths,countriesAndTerritories,popData2019,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,14,12,2020,746,6,Afghanistan,38041757.0,9.013779
1,13,12,2020,298,9,Afghanistan,38041757.0,7.052776
2,12,12,2020,113,11,Afghanistan,38041757.0,6.868768
3,11,12,2020,63,10,Afghanistan,38041757.0,7.134266
4,10,12,2020,202,16,Afghanistan,38041757.0,6.968658
...,...,...,...,...,...,...,...,...
61882,7,4,2020,0,0,Zimbabwe,14645473.0,0.047796
61883,6,4,2020,0,0,Zimbabwe,14645473.0,0.047796
61884,5,4,2020,0,0,Zimbabwe,14645473.0,0.047796
61885,4,4,2020,1,0,Zimbabwe,14645473.0,0.054624


## calcule de cumules de Deaths

In [105]:
#pour chaque ligne on somme les 14  (deaths/Population)*100 000.
baseCovid['Cumulative_number_for_14_days_of_COVID-19_deaths_per_100000']= 0
nbrLignes = len(baseCovid)
column_deaths = 4
column_pop = 6
count=0 
while(count < nbrLignes-14):
    somme = 0 
    row = count
    i=0
    while(i<14):
        somme = somme +baseCovid.iloc[  row , column_deaths ]
        i = i+1
        row = row+1
    cumules = somme/baseCovid.iloc[ count, column_pop]*100000
    baseCovid['Cumulative_number_for_14_days_of_COVID-19_deaths_per_100000']=cumules
    count = count+1
    print("cumules  =" , cumules)

cumules  = 0.5467675954083824
cumules  = 0.5309954532331407
cumules  = 0.5362528339582213
cumules  = 0.5467675954083824
cumules  = 0.5204806917829794
cumules  = 0.5099659303328182
cumules  = 0.5099659303328182
cumules  = 0.5388815243207615
cumules  = 0.49156509779503615
cumules  = 0.49682247852011674
cumules  = 0.5152233110578988
cumules  = 0.5152233110578988
cumules  = 0.4652781941696332
cumules  = 0.3732740314807226
cumules  = 0.3601305796680211
cumules  = 0.3837887929308838
cumules  = 0.3864174832934241
cumules  = 0.3732740314807226
cumules  = 0.4100756965562868
cumules  = 0.38904617365596444
cumules  = 0.35487319894294056
cumules  = 0.341729747130239
cumules  = 0.3285862953175375
cumules  = 0.3128141531422957
cumules  = 0.2523542748038688
cumules  = 0.2392108229911673
cumules  = 0.2549829651664091
cumules  = 0.23132475190354643
cumules  = 0.2234386808159255
cumules  = 0.21292391936576432
cumules  = 0.1813796350152807
cumules  = 0.16560749284003892
cumules  = 0.1366918988520956
cumu

## mediane et reduction des lignes 

In [106]:
baseCovid = baseCovid.groupby('countriesAndTerritories')[[
    'cases',	'deaths',	'popData2019',
    'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000',
    'Cumulative_number_for_14_days_of_COVID-19_deaths_per_100000']].median()

# Diet Dataset

In [109]:
baseDiet = pd.read_csv('./data/baseDiet.csv')
baseDiet

,countryname,female,age,urban,edu,v01_wt_median,v01_lowerci_95,v01_upperci_95,v02_wt_median,v02_lowerci_95,...,v23_upperci_95,v36_wt_median,v36_lowerci_95,v36_upperci_95,v41_wt_median,v41_lowerci_95,v41_upperci_95,v57_wt_median,v57_lowerci_95,v57_upperci_95
0,Afghanistan,0,2,0,1,47.413273,26.237797,84.176018,50.071449,22.658962,...,58.883312,255.40314,134.21686,458.62183,1988.4474,999.12262,3863.7734,103.847820,34.508781,307.00015
1,Afghanistan,0,2,0,2,59.956844,32.680233,110.521920,53.862320,23.907200,...,61.194340,266.46301,144.65649,484.50241,2036.7220,1054.16890,3968.9531,141.322830,49.329239,408.49686
2,Afghanistan,0,2,0,3,70.714760,37.900639,128.872090,60.245506,26.682507,...,61.337482,283.05048,152.73541,509.11826,2105.3442,1082.03920,4248.7339,177.695850,60.637211,535.30798
3,Afghanistan,0,2,0,999,48.884312,26.237797,128.872090,50.671410,22.658962,...,61.337482,257.05252,134.21686,509.11826,1995.4592,999.12262,4248.7339,108.452680,34.508781,535.30798
4,Afghanistan,0,2,1,1,55.521770,30.984436,99.499596,52.511799,24.031862,...,60.872921,268.53934,140.64723,486.56586,1887.2854,962.28314,3679.2844,121.987970,41.345661,385.12958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139855,Zimbabwe,999,999,1,999,89.854965,48.753170,180.243730,118.562550,52.889359,...,141.381180,465.41437,243.28893,971.91095,1329.3700,276.21921,6991.2603,69.526268,34.842648,410.39999
139856,Zimbabwe,999,999,999,1,68.779266,39.879131,112.427360,113.423210,52.889359,...,131.301120,484.62579,255.12506,984.37024,1415.4803,286.49463,7463.6157,40.462524,22.885603,164.87851
139857,Zimbabwe,999,999,999,2,87.059586,50.266865,142.163940,121.887760,56.875553,...,137.029980,476.48224,243.28893,979.20245,1425.1400,276.21921,7350.4268,62.510635,35.240677,253.65860
139858,Zimbabwe,999,999,999,3,109.648410,63.109200,180.243730,137.920650,63.690601,...,141.381180,506.30121,259.24951,1038.48820,1446.4594,289.36285,7644.8521,100.854260,56.493141,410.39999


In [112]:
print(baseDiet['countryname'].isna().sum())
print(baseDiet['female'].isna().sum())
print(baseDiet['age'].isna().sum())
print(baseDiet['urban'].isna().sum())
print(baseDiet['edu'].isna().sum())
print(baseDiet['v01_wt_median'].isna().sum())
print(baseDiet['v01_lowerci_95'].isna().sum())
print(baseDiet['v01_upperci_95'].isna().sum())
print(baseDiet['v02_wt_median'].isna().sum())
print(baseDiet['v02_lowerci_95'].isna().sum())
print(baseDiet['v02_upperci_95'].isna().sum())
print(baseDiet['v05_wt_median'].isna().sum())
print(baseDiet['v05_lowerci_95'].isna().sum())
print(baseDiet['v05_upperci_95'].isna().sum())
print(baseDiet['v06_wt_median'].isna().sum())
print(baseDiet['v06_lowerci_95'].isna().sum())
print(baseDiet['v06_upperci_95'].isna().sum())
print(baseDiet['v10_wt_median'].isna().sum())
print(baseDiet['v10_lowerci_95'].isna().sum())
print(baseDiet['v10_upperci_95'].isna().sum())
print(baseDiet['v15_wt_median'].isna().sum())
print(baseDiet['v15_lowerci_95'].isna().sum())
print(baseDiet['v15_upperci_95'].isna().sum())
print(baseDiet['v16_wt_median'].isna().sum())
print(baseDiet['v16_lowerci_95'].isna().sum())
print(baseDiet['v16_upperci_95'].isna().sum())
print(baseDiet['v23_wt_median'].isna().sum())
print(baseDiet['v23_lowerci_95'].isna().sum())
print(baseDiet['v23_upperci_95'].isna().sum())
print(baseDiet['v23_wt_median'].isna().sum())
print(baseDiet['v36_wt_median'].isna().sum())
print(baseDiet['v36_lowerci_95'].isna().sum())
print(baseDiet['v36_upperci_95'].isna().sum())
print(baseDiet['v41_wt_median'].isna().sum())
print(baseDiet['v41_lowerci_95'].isna().sum())
print(baseDiet['v41_upperci_95'].isna().sum())
print(baseDiet['v57_wt_median'].isna().sum())
print(baseDiet['v57_lowerci_95'].isna().sum())
print(baseDiet['v57_upperci_95'].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [113]:
baseDiet.columns
baseDiet = baseDiet[['countryname', 'female', 'age', 'urban', 'edu', 'v01_wt_median',
       'v01_lowerci_95', 'v01_upperci_95', 'v02_wt_median', 'v02_lowerci_95',
       'v02_upperci_95', 'v05_wt_median', 'v05_lowerci_95', 'v05_upperci_95',
       'v06_wt_median', 'v06_lowerci_95', 'v06_upperci_95', 'v10_wt_median',
       'v10_lowerci_95', 'v10_upperci_95', 'v15_wt_median', 'v15_lowerci_95',
       'v15_upperci_95', 'v16_wt_median', 'v16_lowerci_95', 'v16_upperci_95',
       'v23_wt_median', 'v23_lowerci_95', 'v23_upperci_95', 'v36_wt_median',
       'v36_lowerci_95', 'v36_upperci_95', 'v41_wt_median', 'v41_lowerci_95',
       'v41_upperci_95', 'v57_wt_median', 'v57_lowerci_95', 'v57_upperci_95']]

In [114]:
baseDiet= baseDiet[(baseDiet["female"]==999) & (baseDiet["age"]==999) & (baseDiet["urban"]==999) & (baseDiet["edu"]==999)]

In [116]:
baseDiet= baseDiet.groupby(["countryname"])[['v01_wt_median', 'v01_lowerci_95', 'v01_upperci_95', 'v02_wt_median',
                         'v02_lowerci_95', 'v02_upperci_95', 'v05_wt_median', 'v05_lowerci_95',
                         'v05_upperci_95', 'v06_wt_median', 'v06_lowerci_95', 'v06_upperci_95',
                         'v10_wt_median', 'v10_lowerci_95', 'v10_upperci_95', 'v15_wt_median',
                         'v15_lowerci_95', 'v15_upperci_95', 'v16_wt_median', 'v16_lowerci_95',
                         'v16_upperci_95', 'v23_wt_median', 'v23_lowerci_95', 'v23_upperci_95',
                         'v36_wt_median', 'v36_lowerci_95', 'v36_upperci_95', 'v41_wt_median',
                         'v41_lowerci_95', 'v41_upperci_95', 'v57_wt_median', 'v57_lowerci_95','v57_upperci_95'
                         ]].median()

In [117]:
baseDiet

,v01_wt_median,v01_lowerci_95,v01_upperci_95,v02_wt_median,v02_lowerci_95,v02_upperci_95,v05_wt_median,v05_lowerci_95,v05_upperci_95,v06_wt_median,...,v23_upperci_95,v36_wt_median,v36_lowerci_95,v36_upperci_95,v41_wt_median,v41_lowerci_95,v41_upperci_95,v57_wt_median,v57_lowerci_95,v57_upperci_95
countryname,,,,,,,,,,,,,,,,,,,,,
Afghanistan,68.874123,26.035311,236.632260,164.552050,22.172523,616.672360,67.914627,2.065899,1347.923000,5.604128,...,123.645840,492.74210,133.64653,1156.59050,2102.5591,962.28314,4743.2705,100.501190,22.703156,805.997380
Albania,131.201130,71.326286,190.332890,125.939610,33.498055,212.729110,34.402298,8.218932,97.234978,3.481505,...,150.213090,693.18658,361.57037,1662.13900,2871.1877,618.75970,6404.3848,112.015170,63.964375,339.058500
Algeria,78.959091,38.397392,155.547780,146.949770,32.047287,259.654450,7.611416,0.503104,51.080276,3.618011,...,112.353180,577.96478,149.47310,1274.13100,2241.1853,1019.40570,5050.5371,105.795680,35.757320,517.624150
Angola,92.537315,26.283932,320.292360,91.610558,20.404427,309.228270,8.995745,0.362253,151.550130,5.703275,...,146.004750,522.34668,268.61966,1110.69760,2775.0244,543.63983,15270.5180,33.969471,4.313348,385.254670
Antigua and Barbuda,115.634030,63.272678,218.557300,134.404100,47.359245,279.877780,19.183748,0.802767,258.612430,2.909066,...,173.463150,732.62366,387.84534,1633.19600,2035.1766,903.59290,4850.5410,66.354477,11.857051,629.365970
Argentina,96.659805,57.299763,139.526570,115.148170,48.365490,195.202500,3.339897,0.991580,7.336632,1.018234,...,89.126839,637.67822,528.45532,995.18964,1925.5149,1353.84450,3133.8726,82.005775,24.714846,366.749020
Armenia,84.048851,47.752846,126.754650,89.957993,24.390646,163.491580,12.504264,2.640964,35.382988,21.236649,...,171.165620,705.01428,348.93890,1635.86980,3413.9768,764.09302,7533.9297,63.955746,31.544561,180.926090
Australia,133.204390,57.430786,241.010040,106.300900,36.788719,182.804920,18.026365,8.071528,37.936508,3.816031,...,138.519230,831.93689,674.99371,1238.25220,2739.8867,885.20374,5832.6714,149.304090,45.425007,486.042180
Austria,119.953430,50.243717,199.894970,95.013680,33.282566,152.788500,5.414868,2.122959,11.921396,3.787370,...,110.442490,771.40259,632.68915,1213.62540,3000.2957,948.19824,6282.2446,87.477577,33.706459,319.145260


## Joiture des 2 fichiers Base Covid et Diet

In [121]:
#rename column in the file
DataMerged = baseDiet.merge(baseCovid,how= 'inner',on='countryname')
DataMerged

KeyError: 'countryname'

## Normalisation et encodage

In [122]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
DataMerged['countryname'] = label.fit_transform(DataMerged['countryname'])
from sklearn.preprocessing import StandardScaler


# perform a robust scaler transform of the dataset
trans = StandardScaler()
DataMerged[numerical] = trans.fit_transform(DataMerged[numerical])
# perform a robust scaler transform of the dataset


NameError: name 'DataMerged' is not defined

In [ ]:
baseDiet.to_csv('./data/baseDiet.csv')
baseCovid.to_csv('./data/baseCovid.csv')
DataMerged.to_csv('./data/DataMerged.csv')